# FLATTENING OF CURVE OF CORONAVIRUS CASES

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing required libraries

In [ ]:
import plotly.graph_objects as go

# Reading confirmed cases & total number of available beds.

Here, the data for the total number of available beds all over India is taken from the below pdf in which the data is provided by **CDDEP & Princeton University** in a study published by "**Geetanjali Kapoor, Aditi Sriram, Jyoti Joshi, Ramanan Laxminarayan**".

Link: https://cddep.org/wp-content/uploads/2020/04/State-wise-estimates-of-current-beds-and-ventilators_5Apr2020-AN_gk.pdf

In [ ]:
cases = pd.read_csv("/kaggle/input/covid19-in-india/covid_19_india.csv")
print(cases.shape)

beds = pd.read_csv("/kaggle/input/hospital-beds-data-india/hospital_beds.csv")
print(beds.shape)

# Cleaning the given data as States name differ in a csv file.
Since the given .csv file has different names for same state like "Telangana" -> "Telangana***" or "Telengana". So, replacing the name as "Telangana" only. We have entries in state names like "Unassigned" or "Cases being reassigned to states" so, for now we are going to ignore them and removing those rows from the data.

In [ ]:
cases['State/UnionTerritory'].unique()

In [ ]:
cases['State/UnionTerritory'].replace({"Telengana" : "Telangana", "Telengana***" : "Telangana",
                                        "Telangana***" : "Telangana"}, inplace = True)

cases['State/UnionTerritory'].replace({"Daman & Diu" : "Dadra and Nagar Haveli and Daman and Diu",
                                          "Dadar Nagar Haveli" : "Dadra and Nagar Haveli and Daman and Diu"},
                                         inplace = True)
cases = cases[(cases['State/UnionTerritory'] != 'Unassigned') &
                    (cases['State/UnionTerritory'] != 'Cases being reassigned to states')]
cases['State/UnionTerritory'].unique()

# Removing/Deleting unrequired columns for the ease of our analysis.

In [ ]:
cases.Date = pd.to_datetime(cases.Date, dayfirst=True)

cases.drop(['Sno', 'Time', 'ConfirmedIndianNational', 'ConfirmedForeignNational'], axis = 1, inplace=True)
cases.head()

# Calculating the total active cases per day 

In [ ]:
daily_cases = cases.groupby('Date').sum().reset_index()
daily_cases['Active'] = 1

for val in daily_cases.index:
    if val != 0:
        daily_cases['Active'].loc[val] = daily_cases['Confirmed'].loc[val] - daily_cases['Cured'].loc[val-1] - daily_cases['Deaths'].loc[val-1]
    
daily_cases

# Plotting of the active cases shows us how the cases are increasing day-by-day.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = daily_cases.Date, y = daily_cases.Active, name = 'Active Cases'))

fig.update_layout(title = 'Daily Active Cases', xaxis_title = 'Time', yaxis_title = 'Count (in lakhs)')
fig.show()

# Calculating total active cases per state per day.
In this, we are trying to find the total number of active cases in a state at a particular day so to try to find and analyze the number of available beds we have in a state on a particular day and how much do we need more to accommodate the patients.

In [ ]:
state_daily_cases = cases.sort_values(by=['State/UnionTerritory', 'Date']).reset_index(drop=True)
state_daily_cases['ActiveCases'] = 0

for st in sorted(cases['State/UnionTerritory'].unique()):
    df = state_daily_cases[state_daily_cases['State/UnionTerritory'] == st]
    for i in df.index:
        conf = state_daily_cases['Confirmed'].iloc[i]
        rec = state_daily_cases['Cured'].iloc[i-1]
        death = state_daily_cases['Deaths'].iloc[i-1]
            
        state_daily_cases['ActiveCases'].iloc[i] = conf - rec - death
    state_daily_cases['ActiveCases'].iloc[df.index[0]] = state_daily_cases['Confirmed'].iloc[df.index[0]]


# Plotting active cases per state per day

In [ ]:
fig = go.Figure()
for st in state_daily_cases['State/UnionTerritory'].unique():
    df = state_daily_cases[state_daily_cases['State/UnionTerritory'] == st]
    fig.add_trace(go.Scatter(x = df['Date'], y = df['ActiveCases'], name = st))

fig.update_layout(title = 'Daily Active Cases', xaxis_title = 'Time', yaxis_title = 'Count (in lakhs)')
fig.show()

# Analyzing data for total available beds across India

In [ ]:
total = beds.iloc[35]
print("Total beds available all over India", total)
beds.drop([35], inplace=True)
beds.head()

# Calculating total number of available beds per state on a particular day after the confirmation of occupied cases.

In [ ]:
beds_per_state = state_daily_cases.set_index('State/UnionTerritory').join(beds.set_index('States'))
beds_per_state['AvailableBeds'] = beds_per_state['Total hospital beds'] - beds_per_state['ActiveCases']
beds_per_state

# DataFrame showing the capacity of patients a state can contain on a particular day

In [ ]:
beds_df = beds_per_state[['Date', 'AvailableBeds']]
beds_df = pd.pivot_table(beds_per_state, values = 'AvailableBeds', index = 'Date',
                               columns = beds_per_state.index)
for st in beds_df.columns:
    val = beds[beds['States'] == st]['Total hospital beds']
    beds_df[st].fillna(int(val), inplace=True)
    
beds_df.head()

# If we want the data to be in csv format then,
# beds_df = beds_df.to_csv("beds_data.csv")

# Visualizing the capacity of patients per state
The line graph shows the number of beds available in a state uptill now. There are 3 states - **"Maharashtra, Andhra Pradesh, Bihar"** where the line goes down representing the negative value which indicates that there is an unavailability of that much beds and we need to have that number of beds in our hospital to accommodate each and every patient. 

*This conclusion is purely based on the given data only and does not guarantee us that it is the true value of unavailable beds since we do not have the data of those patients who are in self-isolation in their home or in isolation wards(other than hospitals) or in the railway coaches provided by the Railways. If we have those data as well then we can have a clear picture of available and unavailable beds in a state at a particular day.*

In [ ]:
fig = go.Figure()
for col in beds_df.columns:
    fig.add_trace(go.Scatter(x = beds_df.index, y = beds_df[col], name = col))

fig.update_layout(title = 'Number of available beds statewise', yaxis_title = 'Number of available beds')
fig.show()

If there is anything you found it wrong, please do correct me. I would be greatful to know that. If you like my notebook please do like it and am open to any kind of feed back. Thank You.